In [1]:
# All as One: Adaboost
# Treat All Participants as One
# Try with 10% ... 90% for Training (several iterations for each)

In [2]:
use_key_features = False
shuffle_times = 100
iterations = 10
if use_key_features:
    output_file = 'all_kf_adaboost_results.csv'
else:
    output_file = 'all_af_adaboost_results.csv'

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

# 1 -- Load Data

In [4]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [5]:
data = [data5, data7, data9, data11, data12, data17, data18]
all_data = pd.concat(data, ignore_index=True, sort=True)

In [6]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

all_data = all_data.drop(columns=remove)

In [7]:
# Remove rows where engagment NaN
all_data = all_data[np.isfinite(all_data['engagement'])]

# Remove rows where engagment is -1
all_data = all_data[all_data['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [8]:
# Main Columns
basic_cols = []
for i in all_data.columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [9]:
# Open Face Columns

of_cols = []
for i in all_data.columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [10]:
# Audio Columns

a_cols = []
for i in all_data.columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [11]:
# ROS Columns

ros_cols = []
for i in all_data.columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [12]:
# For Window Only:
non_window_features = []
window_features = []
for i in all_data.columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [13]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [14]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [15]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

all_data = all_data[features_to_keep]

In [16]:
# All as One Participant: Shuffle Data! 

for i in range(shuffle_times):
    all_data.reindex(np.random.permutation(all_data.index))
all_data = all_data.reset_index(drop=True)

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [17]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [18]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing 

# split_size: how much data for training
def split(split_size):
    y_data = all_data['engagement'].copy()
    X_data = all_data.drop(columns=['engagement', 'session_num', 'participant'], axis=1).copy()
        
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=1-split_size, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [19]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Hyperparameter Combinations to Try:  
def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    model = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=100))
    model.fit(X_train.values, y_train.values)
    
    scores = model.predict(X_test.values)
    pred = [round(value) for value in scores] 
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [21]:
all_results = []
for perc in tqdm([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    print('Training Percentage:', perc)
    for n in range(iterations):
        print()
        new_results = {}
        
        # Shuffle Data For This Iteration
        for i in range(shuffle_times):
            all_data.reindex(np.random.permutation(all_data.index))        
        all_data = all_data.reset_index(drop=True)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(perc)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        '''
        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1
        
        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_index, best_acc = None, -1, 0
        for index,h in enumerate(hp_combos):
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_index = index
                best_acc = acc
        '''
        best = {}
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_percentage'] = perc
        results['iteration'] = n
        results['hyperparameters'] = ""
        all_results.append(results)

  0%|          | 0/11 [00:00<?, ?it/s]

Training Percentage: 0.01

Best HP {}
{'f1_1': 0.8681256378274181, 'precision_1': 0.8433575677461996, 'precision_0': 0.7752303182579564, 'auc': 0.7905847010154062, 'recall_1': 0.8943925233644859, 'recall_0': 0.6867768786663266, 'f1_0': 0.7283278212867453, 'accuracy': 0.8224413213444864}

Best HP {}
{'f1_1': 0.8747424206365331, 'precision_1': 0.8282840496867099, 'precision_0': 0.8220376952716628, 'auc': 0.7823194118964911, 'recall_1': 0.9267221764423728, 'recall_0': 0.6379166473506096, 'f1_0': 0.7183670273162709, 'accuracy': 0.8266036689728322}

Best HP {}
{'f1_1': 0.8679093375339109, 'precision_1': 0.8257143491485536, 'precision_0': 0.7980329967701574, 'auc': 0.7752997315018633, 'recall_1': 0.9146489922900041, 'recall_0': 0.6359504707137226, 'f1_0': 0.7078316751190429, 'accuracy': 0.8180700527927103}

Best HP {}
{'f1_1': 0.8631222927668665, 'precision_1': 0.8340328425301592, 'precision_0': 0.7692307692307693, 'auc': 0.7793464396489354, 'recall_1': 0.8943142414479994, 'recall_0': 0.6643

  9%|▉         | 1/11 [00:26<04:20, 26.02s/it]

{'f1_1': 0.8679716587447688, 'precision_1': 0.8368870422792439, 'precision_0': 0.7824434733041992, 'auc': 0.7850115135034517, 'recall_1': 0.9014545141578449, 'recall_0': 0.6685685128490584, 'f1_0': 0.7210375576872569, 'accuracy': 0.8207699539570427}
Training Percentage: 0.05

Best HP {}
{'f1_1': 0.8909775358053756, 'precision_1': 0.8542825230733055, 'precision_0': 0.8432171332130595, 'auc': 0.8156959263228545, 'recall_1': 0.9309664441568758, 'recall_0': 0.7004254084888331, 'f1_0': 0.7652169320552431, 'accuracy': 0.8510982155567279}

Best HP {}
{'f1_1': 0.8894677988178769, 'precision_1': 0.8545321464831299, 'precision_0': 0.8367276393489846, 'auc': 0.8147629315394803, 'recall_1': 0.927381745502998, 'recall_0': 0.7021441175759626, 'f1_0': 0.7635508122601335, 'accuracy': 0.8493564783413302}

Best HP {}
{'f1_1': 0.8904799235650064, 'precision_1': 0.8530809580248072, 'precision_0': 0.8432347093907145, 'auc': 0.8143075379746688, 'recall_1': 0.9313083693103934, 'recall_0': 0.697306706638944, 

 18%|█▊        | 2/11 [11:02<31:23, 209.27s/it]

...

{'f1_1': 0.9467730810917332, 'precision_1': 0.922610131735607, 'precision_0': 0.9412419700214133, 'auc': 0.9086416953835004, 'recall_1': 0.9722357131293509, 'recall_0': 0.84504767763765, 'f1_0': 0.8905547226386807, 'accuracy': 0.9283782350445482}
Training Percentage: 0.8

Best HP {}
{'f1_1': 0.9489890116434888, 'precision_1': 0.9268803443662381, 'precision_0': 0.9426487234310149, 'auc': 0.9142769214853217, 'recall_1': 0.9721781574130567, 'recall_0': 0.8563756855575868, 'f1_0': 0.8974435730108362, 'accuracy': 0.9318669954657545}

Best HP {}
{'f1_1': 0.9482794269679515, 'precision_1': 0.927485988328422, 'precision_0': 0.9366943203573708, 'auc': 0.91199723694328, 'recall_1': 0.9700265893159294, 'recall_0': 0.8539678845706307, 'f1_0': 0.8934201716476962, 'accuracy': 0.930355580303874}

Best HP {}
{'f1_1': 0.9480496348631479, 'precision_1': 0.9256811594202898, 'precision_0': 0.9406994424733908, 'auc': 0.9121355759437703, 'recall_1': 0.9715259187150158, 'recall_0': 0.8527452331725247, 'f1_0'

 91%|█████████ | 10/11 [11:47:26<1:49:54, 6594.28s/it]

{'f1_1': 0.9488990196954799, 'precision_1': 0.9276975064087625, 'precision_0': 0.9405866131862622, 'auc': 0.9145913051561922, 'recall_1': 0.971092272976764, 'recall_0': 0.8580903373356203, 'f1_0': 0.8974466303892843, 'accuracy': 0.9317874472993397}
Training Percentage: 0.9

Best HP {}
{'f1_1': 0.9526574628192215, 'precision_1': 0.9327068105348648, 'precision_0': 0.944585020242915, 'auc': 0.9195015497481889, 'recall_1': 0.9734802615645435, 'recall_0': 0.865522837931834, 'f1_0': 0.9033272837265578, 'accuracy': 0.9364410150346034}

Best HP {}
{'f1_1': 0.9517470881863561, 'precision_1': 0.9318128927158482, 'precision_0': 0.9431732461654514, 'auc': 0.9187181948974735, 'recall_1': 0.9725528297303863, 'recall_0': 0.8648835600645608, 'f1_0': 0.902333413519365, 'accuracy': 0.9354068888712115}

Best HP {}
{'f1_1': 0.9505386584131897, 'precision_1': 0.9323914058851004, 'precision_0': 0.9371100573995508, 'auc': 0.917905752847168, 'recall_1': 0.9694063372587107, 'recall_0': 0.8664051684356253, 'f1_

100%|██████████| 11/11 [14:38:27<00:00, 7694.15s/it]  

{'f1_1': 0.9477728418777763, 'precision_1': 0.9238150965476887, 'precision_0': 0.9456035767511177, 'auc': 0.9134883382584389, 'recall_1': 0.9730062862073215, 'recall_0': 0.8539703903095559, 'f1_0': 0.8974540311173974, 'accuracy': 0.9307930952191552}


In [22]:
results = pd.DataFrame(columns=['train_percentage', 'iteration', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'hyperparameters'])
results = results.append(all_results, ignore_index=True, sort=True)

In [23]:
results.head()

,accuracy,auc,f1_0,f1_1,hyperparameters,iteration,precision_0,precision_1,recall_0,recall_1,train_percentage
0,0.822441,0.790585,0.728328,0.868126,,0,0.775230,0.843358,0.686777,0.894393,0.01
1,0.826604,0.782319,0.718367,0.874742,,1,0.822038,0.828284,0.637917,0.926722,0.01
2,0.818070,0.775300,0.707832,0.867909,,2,0.798033,0.825714,0.635950,0.914649,0.01
3,0.814639,0.779346,0.712970,0.863122,,3,0.769231,0.834033,0.664379,0.894314,0.01
4,0.816326,0.781128,0.715350,0.864421,,4,0.771620,0.835405,0.666729,0.895527,0.01


In [24]:
results.to_csv(output_file, index=False)